In [16]:
import dataloader
from collections import Counter
import datasets
from dataloader.base import DatasetConfig
from dataloader.mdace.constants import MDACE_INPATIENT_PATH as mdace_inpatient
from segmenters.base import factory

In [3]:
SEGMENTER = factory("document", spacy_model="en_core_web_lg")
mdace_config = {
        "identifier": "mdace-icd10cm-3.3",
        "name_or_path": mdace_inpatient,
        "subsets": ["icd10cm-3.3"],
        "options": {"segmenter": SEGMENTER},
    }

In [4]:

mdace: datasets.Dataset = dataloader.load_dataset(DatasetConfig(**mdace_config))

Adapting dataset using MdaceAdapter:   0%|          | 0/331 [00:00<?, ? examples/s]

Adapting dataset using MdaceAdapter:   0%|          | 0/110 [00:00<?, ? examples/s]

Adapting dataset using MdaceAdapter:   0%|          | 0/115 [00:00<?, ? examples/s]

In [44]:
# Creating a Counter
for split in mdace:
    print(f"Split: {split}")
    ctr = Counter(mdace[split]["note_type"])

    # Calculate total counts
    total = sum(ctr.values())

    # Create a dictionary with counts and percentages
    ctr_with_percentage = {key: {"count": value, "percentage": (value / total) * 100} for key, value in ctr.items()}

    # Print the results
    for note_type, stats in ctr_with_percentage.items():
        print(f"Note type: {note_type}, Count: {stats['count']}, Percentage: {stats['percentage']:.2f}%")

Split: train
Note type: Physician, Count: 121, Percentage: 36.56%
Note type: Discharge summary, Count: 184, Percentage: 55.59%
Note type: Nutrition, Count: 7, Percentage: 2.11%
Note type: Nursing, Count: 1, Percentage: 0.30%
Note type: General, Count: 8, Percentage: 2.42%
Note type: Radiology, Count: 6, Percentage: 1.81%
Note type: Rehab Services, Count: 3, Percentage: 0.91%
Note type: Case Management, Count: 1, Percentage: 0.30%
Split: validation
Note type: Discharge summary, Count: 60, Percentage: 54.55%
Note type: Physician, Count: 37, Percentage: 33.64%
Note type: Radiology, Count: 6, Percentage: 5.45%
Note type: Rehab Services, Count: 2, Percentage: 1.82%
Note type: General, Count: 4, Percentage: 3.64%
Note type: Nutrition, Count: 1, Percentage: 0.91%
Split: test
Note type: Radiology, Count: 6, Percentage: 5.22%
Note type: Physician, Count: 38, Percentage: 33.04%
Note type: Discharge summary, Count: 61, Percentage: 53.04%
Note type: General, Count: 5, Percentage: 4.35%
Note type: 

In [28]:
index2code = mdace["train"][0]["index2code"]
train_targets = set(code for row in mdace["train"] for codes in row["targets"] for code in codes)
train_codes = [index2code[str(i)] for i in train_targets]
train_code_lengths = [len(code) for code in train_codes]
train_ctr = Counter(train_code_lengths)
val_targets = set(code for row in mdace["validation"] for codes in row["targets"] for code in codes)
val_codes = [index2code[str(i)] for i in val_targets]
val_code_lengths = [len(code) for code in val_codes]
val_ctr = Counter(val_code_lengths)
test_targets = set(code for row in mdace["test"] for codes in row["targets"] for code in codes)
test_codes = [index2code[str(i)] for i in test_targets]
test_code_lengths = [len(code) for code in test_codes]
test_ctr = Counter(test_code_lengths)

print(f"Train targets: {len(train_targets)}")
print(f"Validation targets: {len(val_targets)}")
print(f"Test targets: {len(test_targets)}")

print(f"Train code lengths: {train_ctr}")
print(f"Validation code lengths: {val_ctr}")
print(f"Test code lengths: {test_ctr}")


Train targets: 682
Validation targets: 294
Test targets: 308
Train code lengths: Counter({5: 283, 6: 217, 7: 153, 3: 29})
Validation code lengths: Counter({5: 114, 6: 97, 7: 69, 3: 14})
Test code lengths: Counter({5: 123, 6: 106, 7: 69, 3: 10})


In [59]:
len(mdace["train"][0]["classes"])

900

In [3]:
mdace_30_config: DatasetConfig = DatasetConfig(**dataloader.DATASET_CONFIGS["mdace-icd10cm-3.0"])
mdace_31_config: DatasetConfig = DatasetConfig(**dataloader.DATASET_CONFIGS["mdace-icd10cm-3.1"])
mdace_32_config: DatasetConfig = DatasetConfig(**dataloader.DATASET_CONFIGS["mdace-icd10cm-3.2"])
mdace_33_config: DatasetConfig = DatasetConfig(**dataloader.DATASET_CONFIGS["mdace-icd10cm-3.3"])

In [5]:
mdace_30: datasets.Dataset = dataloader.load_dataset(mdace_30_config)
mdace_31: datasets.Dataset = dataloader.load_dataset(mdace_31_config)
mdace_32: datasets.Dataset = dataloader.load_dataset(mdace_32_config)
mdace_33: datasets.Dataset = dataloader.load_dataset(mdace_33_config)


In [9]:
dsets = [mdace_30, mdace_31, mdace_32, mdace_33]



Dataset({
    features: ['note_type', 'note_subtype', 'classes', 'aid', 'segments', 'targets', 'fewshots', 'index2code'],
    num_rows: 115
})

In [10]:
set(mdace_30["note_type"])

{'Consult',
 'Discharge summary',
 'ECG',
 'General',
 'Nutrition',
 'Physician',
 'Radiology'}

In [12]:
from collections import Counter

# Creating a Counter
ctr = Counter(mdace_30["note_type"])

# Calculate total counts
total = sum(ctr.values())

# Create a dictionary with counts and percentages
ctr_with_percentage = {key: {"count": value, "percentage": (value / total) * 100} for key, value in ctr.items()}

# Print the results
for note_type, stats in ctr_with_percentage.items():
    print(f"Note type: {note_type}, Count: {stats['count']}, Percentage: {stats['percentage']:.2f}%")


Note type: Radiology, Count: 6, Percentage: 5.22%
Note type: Physician, Count: 38, Percentage: 33.04%
Note type: Discharge summary, Count: 61, Percentage: 53.04%
Note type: General, Count: 5, Percentage: 4.35%
Note type: Consult, Count: 2, Percentage: 1.74%
Note type: Nutrition, Count: 2, Percentage: 1.74%
Note type: ECG, Count: 1, Percentage: 0.87%


In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")

In [7]:
mdace

DatasetDict({
    train: Dataset({
        features: ['note_type', 'note_subtype', 'classes', 'aid', 'segments', 'targets', 'fewshots', 'index2code'],
        num_rows: 331
    })
    validation: Dataset({
        features: ['note_type', 'note_subtype', 'classes', 'aid', 'segments', 'targets', 'fewshots', 'index2code'],
        num_rows: 110
    })
    test: Dataset({
        features: ['note_type', 'note_subtype', 'classes', 'aid', 'segments', 'targets', 'fewshots', 'index2code'],
        num_rows: 115
    })
})

In [8]:
mdace["train"][0]["segments"]

["[**Hospital Unit Name 10**] Resident Admission Note\n   .\n   Reason for MICU Admission:  Pancreatitis and hypotension.\n   .\n   Primary Care Physician: [**Name Initial (NameIs) 324**].\n   .\n   CC:[**CC Contact Info 325**]   .\n   HPI: Ms. [**Known lastname 326**] is an 88 y.o. F with Alzheimer's dementia, afib, and\n   failure to thrive, transferred from [**Hospital 327**] Hospital for emergent ERCP\n   for presumed gallstone pancreatitis.  The patient was at her nursing\n   home when she was noted to have an episode of vomiting of large amount\n   of undigested food at 2 AM on [**2187-9-21**].  She then vomited a large amount\n   of brown liquid that was reportedly guiaic positive.  MD [**First Name (Titles) **] [**Last Name (Titles) 122**] made\n   aware and referred to ED.  O2 sats were noted to be 85-88% on RA and 2\n   L NC brought O2 sat to 92%.\n   .\n   At [**Hospital 327**] Hospital ED, VS: BP 129/68  P 83  R 18  Temp 100.3  O2 sat\n   93% on RA.  EKG reported to have si

In [9]:
def tokenize_function(row: dict):
    note = row["segments"][0]
    row["input_ids"] = tokenizer.encode(note, add_special_tokens=True, truncation=False)
    return row
encoded_mdace = mdace.map(tokenize_function)

Map:   0%|          | 0/331 [00:00<?, ? examples/s]

Map:   0%|          | 0/110 [00:00<?, ? examples/s]

Map:   0%|          | 0/115 [00:00<?, ? examples/s]

In [22]:
from collections import defaultdict
combined_note_lengths = defaultdict(list)
train_note_lengths = defaultdict(list)
for row in encoded_mdace["train"]:
    combined_note_lengths[row["note_type"]].append(len(row["input_ids"]))
avg_train_note_length = {note_type: sum(lengths) / len(lengths) for note_type, lengths in train_note_lengths.items()}
val_note_lengths = defaultdict(list)
for row in encoded_mdace["validation"]:
    combined_note_lengths[row["note_type"]].append(len(row["input_ids"]))
avg_val_note_length = {note_type: sum(lengths) / len(lengths) for note_type, lengths in val_note_lengths.items()}
test_note_lengths = defaultdict(list)
for row in encoded_mdace["test"]:
    combined_note_lengths[row["note_type"]].append(len(row["input_ids"]))
avg_test_note_length = {note_type: sum(lengths) / len(lengths) for note_type, lengths in test_note_lengths.items()}

print(f"Train note lengths: {avg_train_note_length}")
print(f"Validation note lengths: {avg_val_note_length}")
print(f"Test note lengths: {avg_test_note_length}")

Train note lengths: {}
Validation note lengths: {}
Test note lengths: {}


In [26]:
import numpy as np


rows = []
for key, lengths in combined_note_lengths.items():
    avg_length = np.mean(lengths)
    diff = max(lengths) - min(lengths)
    # Format as "average ± difference"
    formatted_length = f"{avg_length:.2f} ± {diff:.2f}"
    print(f"Note type: {key}, Average length: {formatted_length}")
    rows.append([key, formatted_length])

Note type: General, Average length: 824.29 ± 3091.00
Note type: Discharge summary, Average length: 3803.23 ± 11882.00
Note type: Physician, Average length: 2155.57 ± 4455.00
Note type: Nursing, Average length: 1085.00 ± 0.00
Note type: Radiology, Average length: 623.67 ± 1330.00
Note type: Nutrition, Average length: 946.30 ± 1393.00
Note type: Rehab Services, Average length: 1075.60 ± 642.00
Note type: Case Management, Average length: 457.00 ± 0.00
Note type: ECG, Average length: 65.00 ± 0.00
Note type: Consult, Average length: 1351.50 ± 521.00


In [27]:
mdace

DatasetDict({
    train: Dataset({
        features: ['note_type', 'note_subtype', 'classes', 'aid', 'segments', 'targets', 'fewshots', 'index2code'],
        num_rows: 331
    })
    validation: Dataset({
        features: ['note_type', 'note_subtype', 'classes', 'aid', 'segments', 'targets', 'fewshots', 'index2code'],
        num_rows: 110
    })
    test: Dataset({
        features: ['note_type', 'note_subtype', 'classes', 'aid', 'segments', 'targets', 'fewshots', 'index2code'],
        num_rows: 115
    })
})

In [32]:
from collections import defaultdict
combined_note_lengths = defaultdict(list)
train_note_lengths = defaultdict(list)
for row in encoded_mdace["train"]:
    combined_note_lengths[row["note_type"]].append(len(row["targets"][0]))
val_note_lengths = defaultdict(list)
for row in encoded_mdace["validation"]:
    combined_note_lengths[row["note_type"]].append(len(row["targets"][0]))
test_note_lengths = defaultdict(list)
for row in encoded_mdace["test"]:
    combined_note_lengths[row["note_type"]].append(len(row["targets"][0]))


In [33]:
import numpy as np


rows = []
for key, lengths in combined_note_lengths.items():
    avg_length = np.mean(lengths)
    diff = max(lengths) - min(lengths)
    # Format as "average ± difference"
    formatted_length = f"{avg_length:.2f} ± {diff:.2f}"
    print(f"Note type: {key}, Average length: {formatted_length}")
    rows.append([key, formatted_length])

Note type: General, Average length: 1.41 ± 3.00
Note type: Discharge summary, Average length: 9.48 ± 24.00
Note type: Physician, Average length: 1.68 ± 6.00
Note type: Nursing, Average length: 1.00 ± 0.00
Note type: Radiology, Average length: 1.17 ± 2.00
Note type: Nutrition, Average length: 1.70 ± 3.00
Note type: Rehab Services, Average length: 1.60 ± 2.00
Note type: Case Management, Average length: 2.00 ± 0.00
Note type: ECG, Average length: 1.00 ± 0.00
Note type: Consult, Average length: 2.00 ± 2.00


In [34]:
len(mdace["train"][0]["classes"])

900